In [1]:
import ib_insync
from datetime import datetime, timedelta, date
# print(ib_insync.__all__)

In [2]:
from ib_insync import *
util.startLoop()

In [3]:
# 1.0 打開連接
ib = IB()
ib.connect('127.0.0.1', port=4002, clientId=1, timeout=5000)

<IB connected to 127.0.0.1:4002 clientId=1>

In [4]:
# 2.0 帳戶資金
[v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE']

[AccountValue(account='DU365095', tag='NetLiquidationByCurrency', value='2205170.3697', currency='BASE', modelCode='')]

In [5]:
# 3.0 期貨
date1 = date.today().strftime('%Y%m')
future1 = Future('NQ', date1, 'GLOBEX')

In [6]:
# 4.0 歷史數據
bars1 = ib.reqHistoricalData(future1, endDateTime='', durationStr='30 D', 
                             barSizeSetting='1 min', whatToShow='MIDPOINT', useRTH=True)
util.df(bars1)

,date,open,high,low,close,volume,average,barCount
0,2020-10-21 21:30:00,11669.500,11684.125,11656.000,11668.250,-1,-1.0,-1
1,2020-10-21 21:31:00,11668.250,11688.125,11665.500,11684.375,-1,-1.0,-1
2,2020-10-21 21:32:00,11684.375,11686.000,11672.125,11681.875,-1,-1.0,-1
3,2020-10-21 21:33:00,11681.875,11696.500,11675.625,11690.375,-1,-1.0,-1
4,2020-10-21 21:34:00,11690.375,11698.000,11686.125,11693.000,-1,-1.0,-1
...,...,...,...,...,...,...,...,...
12835,2020-12-03 05:55:00,12461.500,12462.875,12460.625,12462.875,-1,-1.0,-1
12836,2020-12-03 05:56:00,12462.875,12463.000,12462.500,12463.000,-1,-1.0,-1
12837,2020-12-03 05:57:00,12463.000,12463.875,12462.750,12462.875,-1,-1.0,-1
12838,2020-12-03 05:58:00,12462.875,12464.000,12462.875,12462.875,-1,-1.0,-1


In [7]:
# 5.0 買盤
order1 = LimitOrder('BUY', 1, 12474.00)
trade1 = ib.placeOrder(future1, order1)
trade1

Trade(contract=Future(symbol='NQ', lastTradeDateOrContractMonth='202012', exchange='GLOBEX'), order=LimitOrder(orderId=14, clientId=1, action='BUY', totalQuantity=1, lmtPrice=12474.0), orderStatus=OrderStatus(orderId=14, status='PendingSubmit', filled=0, remaining=0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2020, 12, 3, 9, 2, 51, 933101, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

In [8]:
# 6.0 倉位
ib.positions()

[Position(account='DU365095', contract=Future(conId=383974419, symbol='NQ', lastTradeDateOrContractMonth='20201218', multiplier='20', currency='USD', localSymbol='NQZ0', tradingClass='NQ'), position=1.0, avgCost=249412.1)]

In [9]:
# 關閉連接
# ib.disconnect()